This python code has been developped by [Taïs Grippa](https://github.com/tgrippa) (Université Libre de Bruxelles). 

Code developped on Ubuntu 22.04 (Ubuntu Jammy) and GRASS GIS 8.0.2 using the Docker environment [available here](https://github.com/tgrippa/Weaksupervision_Vaihingen).

# Define working environment

**Import libraries**

In [1]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import glob
import math
import pickle
import time 
print(sys.version)

3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]


In [2]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

**Add folder with SCR provided belong to this notebook**

In [3]:
# Add local module to the path
src = os.path.abspath(os.path.join(os.environ['HOME'],'github','SRC'))
if src not in sys.path:
    sys.path.append(src)

**Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev)**

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [4]:
exec(open(os.path.join(os.environ['HOME'],'github','SRC', 'config.py')).read())

In [5]:
print(config_parameters)

{'GISBASE': '/usr/lib/grass78', 'PYTHONLIB': '/usr/bin/python3', 'gisdb': '/home/tais/GRASSDATA', 'location': 'flair-one', 'permanent_mapset': 'PERMANENT', 'locationepsg': '2154', 'outputfolder': '/home/tais/result', 'inputdir': '/home/tais/data'}


In [6]:
print(data)

{'legend': '/home/tais/github/Legend.txt'}


In [7]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [8]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

PATH	= /.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/grass78/bin:/usr/lib/grass78/script:/usr/lib/grass78/lib 	
HOSTNAME	= d35fee4c10cd 	
DISPLAY	= unix 	
LANG	= C.UTF-8 	
LC_ALL	= C.UTF-8 	
JUPYTER_ENABLE_LAB	= yes 	
TINI_VERSION	= v0.6.0 	
HOME	= /home/tais 	
GIT_PYTHON_REFRESH	= quiet 	
PYDEVD_USE_FRAME_EVAL	= NO 	
JPY_SESSION_NAME	= /home/tais/github/GRASSGIS/notebook/B_OBIA_RuleBased.ipynb 	
JPY_PARENT_PID	= 7 	
TERM	= xterm-color 	
CLICOLOR	= 1 	
PAGER	= cat 	
GIT_PAGER	= cat 	
MPLBACKEND	= module://matplotlib_inline.backend_inline 	
PYTHONPATH	= :/usr/lib/grass78/etc/python:/usr/lib/grass78/etc/python/grass:/usr/lib/grass78/etc/python/grass/script 	
LD_LIBRARY_PATH	= :/usr/lib/grass78/lib 	
GISBASE	= /usr/lib/grass78 	
PYTHONLIB	= /usr/bin/python3 	
GIS_LOCK	= $$ 	
GISRC	= /home/tais/.grass7/rc 	


**GRASS GIS Python libraries**

In [9]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

**Other functions**

In [10]:
from grass.script import vector
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function for sorting string number naturally
from sorting_natural import natural_keys

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [11]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Init
    gsetup.init(config_parameters['GISBASE'], config_parameters['gisdb'], config_parameters['location'], mapset)
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

In [12]:
def segmentation(mapset):
    # Image ID
    image_id = int(mapset.split('_')[-1])
    # Define computational region
    gscript.run_command('g.region', overwrite=True, raster='red', save="region")
    # Define group of layers
    gscript.run_command('g.remove', quiet=True, flags='f', type='group', name='top')
    gscript.run_command('i.group', quiet=True, group='top', input="nir,red,green,blue")
    # gscript.run_command('i.group', group='top', input="nir,red,green,blue,ndsm") # TODO: Test of nDSM in the segmentation  
    # Unsupervised Segmentation parameter optimization 
    gscript.run_command('i.segment.uspo', quiet=True, overwrite=True, group='top', segment_map='best_segment', regions='region', segmentation_method='region_growing', 
                        threshold_start='0.005', threshold_stop='0.05', threshold_step='0.002', minsizes='50', memory='3000', processes='25')
    # Trick the segment ID to be sure the ID is unique through all the images
    #formula = "segmentation = (%s*1000000) + best_segment_region_rank1"%image_id
    formula = "segmentation = best_segment_region_rank1"
    gscript.mapcalc(formula, quiet=True, overwrite=True)
    # Exportation
    gscript.run_command('r.out.gdal', quiet=True, overwrite=True, flags='m', input='segmentation', 
                        output=os.path.join(config_parameters['outputfolder'],'segment_rast','segment_%s.tiff'%mapset), format='GTiff')

In [55]:
def compute_stats(mapset):
    # Define computational region
    gscript.run_command('g.region', overwrite=True, raster='segmentation')
    # Define list of raster on which to compute statistics
    rast_layers=[]
    rast_layers.append('red')
    rast_layers.append('green')
    rast_layers.append('blue')
    rast_layers.append('nir')
    rast_layers.append('ndvi')
    rast_layers.append('ndsm')
    rast_layers.append('text_green_DE')
    rast_layers.append('text_green_Entr')
    rast_layers.append('text_red_ASM')
    rast_layers.append('text_red_IDM')
    rast_layers.append('text_nir_DE')

    try:
        # Compute segment statistics
        gscript.run_command('i.segment.stats', quiet=True, overwrite=True, map='segmentation',
                            rasters=','.join(rast_layers), raster_statistics='stddev,coeff_var,sum,median,perc_90', 
                            area_measures='area,perimeter,compact_circle,compact_square,fd',
                            csvfile=os.path.join(config_parameters['outputfolder'],'stats', 'stats_%s.csv'%mapset),
                            vectormap='segment',
                            processes='20')   
    except:
        #print("execution of i.segment.stats failed using Python run_command function. Now try with bash script")
        # Create a temporary file
        with tempfile.NamedTemporaryFile(suffix=".sh", delete=False) as temp_file:
            # Write the bash script content to the temporary file
            temp_file.write(b'#!/bin/bash\n')
            content = 'grass /home/tais/GRASSDATA/flair-one/%s --exec i.segment.stats --o --q map=segmentation rasters=%s \
            raster_statistics=stddev,coeff_var,sum,median,perc_90 area_measures=area,perimeter,compact_circle,compact_square,fd \
            csvfile=%s vectormap=segment processes=20\n'%(mapset,','.join(rast_layers),os.path.join(config_parameters['outputfolder'],'stats', 'stats_%s.csv'%mapset)) 
            temp_file.write(content.encode())
        
            # Close the temporary file
            temp_file.close()
            # Get the path of the temporary file
            temp_file_path = temp_file.name
        
            try:
                # Add execution permissions to the temporary file
                os.chmod(temp_file_path, 0o755)
                # Execute the temporary file
                stdout = subprocess.run(['bash', temp_file_path])
                #print(stdout)        
            
            except:
                print("i.segment.stats using bash script failed too. Please check")
        
            finally:
                # Clean up the temporary file
                os.remove(temp_file_path)
    # Export vector layer as GPKG
    gscript.run_command('v.out.ogr', quiet=True, overwrite=True, input='segment', 
                        output=os.path.join(config_parameters['outputfolder'],'segment_vect', 'segment_%s.gpkg'%mapset), format='GPKG')

In [47]:
import subprocess

def worker(mapset):
    # Launch a new mapset for this image
    launch_mapset(mapset)
    # Segmentation
    segmentation(mapset)
    # Compute statistics
    compute_stats(mapset)

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create new directories

In [48]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder'])
check_create_dir(os.path.join(config_parameters['outputfolder'],'stats'))
check_create_dir(os.path.join(config_parameters['outputfolder'],'segment_rast'))
check_create_dir(os.path.join(config_parameters['outputfolder'],'segment_vect'))

The folder '/home/tais/result' already exists
The folder '/home/tais/result/stats' already exists
The folder '/home/tais/result/segment_rast' already exists
The folder '/home/tais/result/segment_vect' already exists


# Segmentation (region growing)

In [49]:
import fnmatch

def find_files(path, pattern):
    for root, dirs, files in os.walk(path):
        for file in fnmatch.filter(files, pattern):
            yield os.path.join(root, file)

In [50]:
# Get a list of path to train images
train_folder = os.path.join(config_parameters['inputdir'],'train')
train_tupple = [(os.path.split(x)[-1].split('.tif')[0],x) for x in find_files(train_folder, "IMG_*.tif")]
print(f'There are {len(train_tupple)} train images')

There are 2950 train images


In [51]:
list_mapsets = os.listdir(os.path.join(config_parameters['gisdb'],config_parameters['location']))
list_mapsets.remove('PERMANENT')

In [52]:
print(f'There are {len(list_mapsets)} mapsets')

There are 2950 mapsets


In [ ]:
# Launch processes in parallel
start_parallel = start_processing()
ncores = 10
p = Pool(ncores)
output = p.map(worker, list_mapsets[:])  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
p.close()
p.join()
# Print
print_processing_time(start_parallel, "Computation (on %s cores) achieved in "%(ncores))

ERROR: Unable to make mapset element .tmp/d35fee4c10cd
       (/home/tais/GRASSDATA/flair-one/IMG_027654/.tmp/d35fee4c10cd): File
       exists
ERROR: Unable to make mapset element .tmp/d35fee4c10cd
       (/home/tais/GRASSDATA/flair-one/IMG_027654/.tmp/d35fee4c10cd): File
       exists
ERROR: Unable to make mapset element .tmp/d35fee4c10cd
       (/home/tais/GRASSDATA/flair-one/IMG_031426/.tmp/d35fee4c10cd): File
       exists
ERROR: ERROR: Unable to make mapset element .tmp/d35fee4c10cd
       (/home/tais/GRASSDATA/flair-one/IMG_031426/.tmp/d35fee4c10cd): File
       exists
Unable to make mapset element .tmp/d35fee4c10cd
       (/home/tais/GRASSDATA/flair-one/IMG_031426/.tmp/d35fee4c10cd): File
       exists
ERROR: Unable to make mapset element .tmp/d35fee4c10cd
       (/home/tais/GRASSDATA/flair-one/IMG_031936/.tmp/d35fee4c10cd): File
       exists
ERROR: Unable to make mapset element .tmp/d35fee4c10cd
       (/home/tais/GRASSDATA/flair-one/IMG_031936/.tmp/d35fee4c10cd): File
      